# 펭귄 몸무게 예측 경진대회 베이스라인

이번 대회는 주어진 펭귄의 종, 부리 길이, 성별 등의 정보들을 통해 몸무게를 예측하는 대회입니다.

언뜻보면 간단해 보이지만 처음 데이터 분석을 접하시는 분들에게는 어디부터 어떻게 시작해야 하는지 막막하실 겁니다.

## 누구나 할 수 있는 데이터 분석

딥러닝, 머신러닝, 모델, 학습, 회귀분석 .... 다 몰라도 괜찮습니다!

누구나 할 수 있는 방식으로 간단하게 데이터 분석 & 예측을 진행해보겠습니다.

# 데이터 불러오기

먼저 분석하려는 데이터를 작업장으로 가져오는 작업이 필요합니다.

이를 위해서 파이썬 라이브러리 중 하나인 Pandas를 이용합니다.

pandas 라이브러리는 엑셀과 같은 행과 열로 이루어진 테이블(table) 형식의 데이터를 편하게 다루도록 해주는 라이브러리입니다.

데이터 분석에는 반드시 사용된다고 해도 과언이 아닌 라이브러리이니, 시간이 날때마다 공부해둡시다.

In [1]:
import pandas as pd

# csv 형식으로 된 데이터 파일을 읽어옵니다.
df_train = pd.read_csv('dataset/train.csv')

# 데이터의 최상단 5 줄을 표시합니다.
df_train.head()

,id,Species,Island,Clutch Completion,Culmen Length (mm),Culmen Depth (mm),Flipper Length (mm),Sex,Delta 15 N (o/oo),Delta 13 C (o/oo),Body Mass (g)
0,0,Gentoo penguin (Pygoscelis papua),Biscoe,Yes,50.0,15.3,220,MALE,8.30515,-25.19017,5550
1,1,Chinstrap penguin (Pygoscelis antarctica),Dream,No,49.5,19.0,200,MALE,9.63074,-24.34684,3800
2,2,Gentoo penguin (Pygoscelis papua),Biscoe,Yes,45.1,14.4,210,FEMALE,8.51951,-27.01854,4400
3,3,Gentoo penguin (Pygoscelis papua),Biscoe,Yes,44.5,14.7,214,FEMALE,8.20106,-26.16524,4850
4,4,Gentoo penguin (Pygoscelis papua),Biscoe,No,49.6,16.0,225,MALE,8.38324,-26.84272,5700


출력된 데이터의 모양을 살펴보면 총 11개의 열(column)이 존재하는 것을 확인할 수 있습니다.

데이터를 자세히 살펴보는 것은 EDA 글을 통해 더 자세히 배워봅시다.

여기에서는 분석 방법에 초점을 맞추겠습니다.

## 결측치 확인

결측치(NA: Not Available)란 값이 누락된 데이터를 말합니다.

보다 정확한 분석을 하기 위해서는 데이터의 결측치를 확인하고 적절히 처리해주어야 합니다.

이번 데이터에 결측치가 있나 확인해볼까요?

In [2]:
df_train.columns

Index(['id', 'Species', 'Island', 'Clutch Completion', 'Culmen Length (mm)',
       'Culmen Depth (mm)', 'Flipper Length (mm)', 'Sex', 'Delta 15 N (o/oo)',
       'Delta 13 C (o/oo)', 'Body Mass (g)'],
      dtype='object')

In [3]:
sum(df_train["Sex"].isna())

3

In [14]:
df_train.dtypes

id                       int64
Species                 object
Island                  object
Clutch Completion       object
Culmen Length (mm)     float64
Culmen Depth (mm)      float64
Flipper Length (mm)      int64
Sex                     object
Delta 15 N (o/oo)      float64
Delta 13 C (o/oo)      float64
Body Mass (g)            int64
dtype: object

In [16]:
dict = []
dict.append("gg")
dict

['gg']

In [20]:
dict1 = []
dict1.append(["gg", "bla_bla"])
dict1

[['gg', 'bla_bla']]

In [21]:
dict1.append(["gg1", "bla_bla1"])
dict1

[['gg', 'bla_bla'], ['gg1', 'bla_bla1']]

In [3]:
def check_missing_col(dataframe):
    missing_col = []
    counted_missing_col = 0
    for i, col in enumerate(dataframe.columns):
        missing_num = sum(dataframe[col].isna())
        is_missing = True if missing_num >= 1 else False
        if is_missing:
            counted_missing_col += 1
            print(f'결측치가 있는 컬럼은: {col}입니다')
            print(f'해당 컬럼에 총 {missing_num}개의 결측치가 존재합니다.')
            missing_col.append([col, dataframe[col].dtype])
            # print(missing_col)
    if counted_missing_col == 0:
        print('결측치가 존재하지 않습니다')
    return missing_col

missing_col = check_missing_col(df_train)

결측치가 있는 컬럼은: Sex입니다
해당 컬럼에 총 3개의 결측치가 존재합니다.
결측치가 있는 컬럼은: Delta 15 N (o/oo)입니다
해당 컬럼에 총 3개의 결측치가 존재합니다.
결측치가 있는 컬럼은: Delta 13 C (o/oo)입니다
해당 컬럼에 총 3개의 결측치가 존재합니다.


### 결측치 처리

이번 데이터에는 결측치가 존재하네요.

원활한 분석을 위해 적절하게 결측치들을 처리해 줍시다.

In [5]:
# dict of list
import pandas as pd
import numpy as np
df = pd.DataFrame({
    'name' : ['A', 'B', 'C'],
    'age' : [25, 15, np.NAN],
    'phone':['010-5556-9898', '010-9876-8877', '010-9876-9097']
}
)

df

,name,age,phone
0,A,25.0,010-5556-9898
1,B,15.0,010-9876-8877
2,C,NaN,010-9876-9097


In [9]:
df.isna().sum()

name     0
age      1
phone    0
dtype: int64

In [10]:
df.isna().sum(axis=1) > 0

0    False
1    False
2     True
dtype: bool

In [11]:
cond = df.isna().sum(axis=1) > 0
df_cond = df.loc[cond]
df_cond

,name,age,phone
2,C,NaN,010-9876-9097


In [34]:
# 결측치가 있는 row들을 확인합니다.
df_train.loc[df_train.isna().sum(axis=1) > 0]

,id,Species,Island,Clutch Completion,Culmen Length (mm),Culmen Depth (mm),Flipper Length (mm),Sex,Delta 15 N (o/oo),Delta 13 C (o/oo),Body Mass (g)
6,6,Adelie Penguin (Pygoscelis adeliae),Torgersen,Yes,42.0,20.2,190,NaN,9.13362,-25.09368,4250
8,8,Adelie Penguin (Pygoscelis adeliae),Torgersen,Yes,34.1,18.1,193,NaN,NaN,NaN,3475
18,18,Adelie Penguin (Pygoscelis adeliae),Dream,No,39.8,19.1,184,MALE,NaN,NaN,4650
70,70,Gentoo penguin (Pygoscelis papua),Biscoe,Yes,46.2,14.4,214,NaN,8.24253,-26.81540,4650
109,109,Adelie Penguin (Pygoscelis adeliae),Torgersen,Yes,36.6,17.8,185,FEMALE,NaN,NaN,3700


결측치들에 대해서 어떤 특별한 패턴은 보이지 않네요.

카테고리형 데이터에 대해서는 행을 삭제하고

수치형 데이터에 대해서는 평균값을 채워주겠습니다.

In [35]:
missing_col

[['Sex', dtype('O')],
 ['Delta 15 N (o/oo)', dtype('float64')],
 ['Delta 13 C (o/oo)', dtype('float64')]]

In [12]:
missing_col[0].dtype()

AttributeError: 'list' object has no attribute 'dtype'

In [16]:
df_train["Delta 15 N (o/oo)"].mean()

8.73763414414414

In [17]:
# 결측치를 처리하는 함수를 작성합니다.
def handle_na(data, missing_col):
    temp = data.copy()
    for col, dtype in missing_col:
        print(dtype)
        if dtype == "object": # dtype == 'O':
            # 카테고리형 feature가 결측치인 경우 해당 행들을 삭제해 주었습니다.
            temp = temp.dropna(subset=[col]) 
        elif dtype == "I" or dtype == "float64":# I is integer
            # 수치형 feature가 결측치인 경우 평균값을 채워주었습니다.
            temp.loc[:,col] = temp[col].fillna(temp[col].mean())
    return temp

data = handle_na(df_train, missing_col)

# # 결측치 처리가 잘 되었는지 확인해 줍니다.
missing_col = check_missing_col(data) 
# still have 

object
float64
float64
결측치가 존재하지 않습니다


# 데이터 설명 방정식 만들기

데이터 분석이란, 주어진 데이터를 분석하여 데이터들 사이의 관계를 알아내고,

이 관계를 통해서 주어지지 않은 데이터를 예측하는 일입니다.

이 말을 잘 생각해보면 데이터 분석이란 방정식을 푸는 일과 똑같다는 걸 알 수 있습니다.

X, Y를 이용해서 만드는 가장 기본적인 방정식을 떠올려봅시다.

## Y = aX + b

여기서 X는 주어진 데이터를 의미하며,  Y는 예측하려는 데이터를 의미합니다.

이번 대회에서는 X란 [펭귄의 종, 부리 길이, 성별 등의 정보]이며, Y는 [펭귄의 몸무게]입니다.

우리의 목표는 X에 어떤 a를 곱하고, 어떤 b를 더해야 정확하게 Y를 구할 수 있는지, 그 a와 b를 찾는 것입니다.

먼저 완전히 주어진 데이터인 train.csv를 이용하여 데이터들 사이의 관계를 파악해봅시다.

## 좀 더 단순하게

먼저 문제를 단순하게 생각해봅시다.

X = [펭귄의 날개 길이] 만을 이용해서 

Y = [펭귄의 몸무게] 를 예측해봅시다.

In [4]:
df_train.columns

Index(['id', 'Species', 'Island', 'Clutch Completion', 'Culmen Length (mm)',
       'Culmen Depth (mm)', 'Flipper Length (mm)', 'Sex', 'Delta 15 N (o/oo)',
       'Delta 13 C (o/oo)', 'Body Mass (g)'],
      dtype='object')

In [5]:
# X와 Y를 각각 따로 저장합니다.
X = df_train['Flipper Length (mm)']
Y = df_train['Body Mass (g)']

X

0      220
1      200
2      210
3      214
4      225
      ... 
109    185
110    190
111    192
112    192
113    228
Name: Flipper Length (mm), Length: 114, dtype: int64

## a 구하기

먼저 날개의 길이와 몸무게가 어떤 관계를 갖는지, 그 평균 값을 구해봅니다.

이 과정은 위 방정식의 a 값을 구하는 것으로 생각할 수 있습니다.

In [6]:
import numpy as np

# 날개의 길이와 몸무게의 평균을 구합니다.
X_mean = np.mean(X)
Y_mean = np.mean(Y)

# 두 값을 나누어 주어서 날개의 길이와 몸무게의 관계를 살펴봅니다.
print(f"날개 길이의 평균:{X_mean:.2f} \n몸무게의 평균:{Y_mean:.2f}")
print(f"따라서 몸무게는 날개 길이의 평균 {Y_mean/X_mean:.2f}배 입니다.")

날개 길이의 평균:203.05 
몸무게의 평균:4327.85
따라서 몸무게는 날개 길이의 평균 21.31배 입니다.


이렇게 보면 우리가 구하고자 하는 Y 는 X의 21 배 정도의 크기를 가지고 있다고 생각할 수 있습니다.

즉, a = 21.31 로 생각할 수 있습니다.

Y = 21.31 X + b

## b 구하기

이번에는 b를 구해봅시다.

a는 X와 Y의 강력한 관계를 나타냈다면, b는 그보다는 약한 관계를 나타냅니다. 

a 만으로는 설명하지 못하는 부분을 구하는 과정이라고 볼 수 있습니다.

a를 넣어서 새롭게 업데이트 된 방정식을 사용하면 b를 구할 수 있습니다.

Y - 21.31 X = b

In [18]:
# b = Y - 21.31 X 를 그대로 작성합니다. 여기서도 평균값을 이용합니다.
b = Y_mean - 21.31 * X_mean

print(f"업데이트 된 방정식의 b는 {b:.2f} 입니다.")

업데이트 된 방정식의 b는 0.80 입니다.


## Y = 21.31 X + 0.8

이렇게 날개 길이를 통해서 몸무게를 예측할 수 있는 방정식을 찾아냈습니다.

이 식을 통해 결과 값들을 예측해봅시다.

In [19]:
predict_Y = 21.31 * X + 0.8
predict_Y

0      4689.00
1      4262.80
2      4475.90
3      4561.14
4      4795.55
        ...   
109    3943.15
110    4049.70
111    4092.32
112    4092.32
113    4859.48
Name: Flipper Length (mm), Length: 114, dtype: float64

# 예측 결과 평가하기

예측값을 만들었다고 데이터 분석이 끝나지는 않습니다.

우리가 예측한 값이 얼마나 정확한지 평가하는 과정을 통해 어떤 부분이 부족한지, 

어떤 부분을 개선해야 더 정확한 예측이 가능한지 살펴봐야 합니다.

이번 대회의 평가 지표(metric)는 RMSE, Root Mean of Squared Error 입니다.

RMSE는 우리가 예측한 값과 실제 값이 얼마나 차이가 나는지, 그 차이값들의 제곱값들의 평균, 그 평균의 square root 값을 취한 것입니다.

단순한 차이값이라고 하면 음수 값, 양수 값이 모두 나올 수 있기 때문에

이 차이값(error)을 제곱하여(square) 평균(mean)을 낸 뒤, 값을 작게 보여주기 위해 root를 취한 것이 것이 RMSE 입니다.

In [20]:
# 대회 규칙의 평가 산식 함수를 그대로 사용합니다.
import numpy as np

def RMSE(true, pred):
    score = np.sqrt(np.mean(np.square(true-pred)))
    return score

In [21]:
# 실제 정답 값을 준비합니다.
real_answer = Y.copy()

# 정답과 예측 값을 함수에 넣어 결과를 확인합니다.
error = RMSE(real_answer, predict_Y)

print(f"이 방정식의 평균 에러는 {error:.2f} 입니다.")

이 방정식의 평균 에러는 532.84 입니다.


# test_data 예측하기

지금까지 train.csv를 통해서 데이터들 사이의 관계를 파악했습니다.

그 결과 아래와 같은 식을 얻을 수 있었습니다.

*펭귄의 몸무게 = 21.31 * 펭귄의 날개 길이 + 0.8*

이제 이 관계를 이용해서 실제로 주어지지 않은 데이터를 예측해봅시다.

In [22]:
# 우리가 예측하고자 하는 test 데이터를 불러옵니다.
df_test = pd.read_csv('dataset/test.csv')

df_test.head()

,id,Species,Island,Clutch Completion,Culmen Length (mm),Culmen Depth (mm),Flipper Length (mm),Sex,Delta 15 N (o/oo),Delta 13 C (o/oo)
0,0,Chinstrap penguin (Pygoscelis antarctica),Dream,Yes,52.0,20.7,210.0,MALE,9.43146,-24.68440
1,1,Gentoo penguin (Pygoscelis papua),Biscoe,Yes,55.9,17.0,228.0,MALE,8.31180,-26.35425
2,2,Adelie Penguin (Pygoscelis adeliae),Dream,Yes,38.9,18.8,190.0,FEMALE,8.36936,-26.11199
3,3,Chinstrap penguin (Pygoscelis antarctica),Dream,Yes,45.2,16.6,191.0,FEMALE,9.62357,-24.78984
4,4,Adelie Penguin (Pygoscelis adeliae),Biscoe,No,37.9,18.6,172.0,FEMALE,8.38404,-25.19837


test 데이터에는 보다시피 **Body Mass (g)** 가 존재하지 않습니다.

하지만 우리는 위에서 **Flipper Length (mm)**를 이용해서 **Body Mass (g)**를 예측할 수 있는 방정식을 구했습니다. 

Y = 21.31 X + 0.8

이제 이 식을 사용하여 test 데이터를 채워봅시다.

In [23]:
# 예측의 재료를 가져옵니다.
test_X = df_test['Flipper Length (mm)']

# 예측을 진행합니다.
predict_test = 21.31 * test_X + 0.8

# 예측 결과를 확인합니다.
predict_test

0      4475.90
1      4859.48
2      4049.70
3      4071.01
4      3666.12
        ...   
223    4326.73
224    4625.07
225    4475.90
226    4284.11
227    4134.94
Name: Flipper Length (mm), Length: 228, dtype: float64

예측 결과가 잘 생성된 것을 확인할 수 있습니다.

# dacon 대회에 제출하기

이제 이 예측 결과를 submission.csv 파일로 만들어서 대회 페이지에 제출해보도록 합시다.

제출한 뒤 리더보드를 통해 결과를 확인합시다.

In [24]:
# 제출용 sample 파일을 불러옵니다.
submission = pd.read_csv('dataset/sample_submission.csv')
submission.head()

,id,Body Mass (g)
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0


In [25]:
# 위에서 구한 예측값을 그대로 넣어줍니다.
submission['Body Mass (g)'] = predict_test

# 데이터가 잘 들어갔는지 확인합니다.
submission

,id,Body Mass (g)
0,0,4475.90
1,1,4859.48
2,2,4049.70
3,3,4071.01
4,4,3666.12
...,...,...
223,223,4326.73
224,224,4625.07
225,225,4475.90
226,226,4284.11


In [26]:
# submission을 csv 파일로 저장합니다.
# index=False란 추가적인 id를 부여할 필요가 없다는 뜻입니다. 
# 정확한 채점을 위해 꼭 index=False를 넣어주세요.
submission.to_csv("./result/submission.csv", index=False)

이렇게 생성된 submission.csv 파일을 데이콘 대회 페이지에 업로드 & 제출하여 결과를 확인하세요

# 축하합니다! 데이터 분석을 완료하셨습니다!

데이터 분석의 첫걸음을 떼신 것을 축하드립니다.

보셨다시피 데이터 분석은 그렇게 어렵지 않습니다.

데이터들 사이의 관계를 찾아서, 그 관계를 통해 주어지지 않은 정보를 예측할 뿐입니다.

고도의 기술을 사용하면 이 예측값이 조금 더 정확해질뿐, 기본적인 개념은 바뀌지 않습니다.

앞으로도 데이콘과 함께 즐겁게 데이터 분석 능력을 키워가시면 좋겠습니다.

감사합니다.

# (한 걸음 더) 방정식으로 모델 학습 맛보기

머신러닝, 딥러닝을 얼핏 들어보면 "모델 학습"이라는 용어가 자주 등장합니다.

모델 학습이란 무엇일까요?

우리가 위에서 구한 방정식으로 모델 학습을 이해해봅시다.

**Flipper Length (mm)**를 이용해서 **Body Mass (g)**를 구한 방정식을 이용해보도록 하겠습니다.

Y = 21.31 X + 0.8

위 식을 이용했을 때 평균 에러는 532.84 이었습니다.

이보다 더 적은 에러를 내는, 더 정확한 식은 어떻게 찾을 수 있을까요?

이 방정식의 a와 b를 변화시키면서 더 정확한 식을 찾아봅시다.

In [ ]:
# 탐색할 a와 b의 후보들을 마련합니다.
a_list = [21, 21.1, 21.2, 21.3, 21.4, 21.5] # 0.1 단위로 변화
b_list = [0.4, 0.5, 0.6, 0.7, 0.8, 0.9] # 0.1 단위로 변화

In [ ]:
# 각각의 후보자들 중에서 가장 에러가 적은 후보를 고르는 함수를 작성합니다.
def find_best_params(a_list, b_list, X, Y):
    min_error = 99999999

    for a in a_list:
        for b in b_list:
            predict = a * X + b
            error = RMSE(Y, predict)
            
            if error < min_error:
                print("모델 개선이 가능합니다.")
                print(f"a={a}, b={b}")
                print(f"error={error:.5f}")
                min_error = error
                best_a, best_b = a,b
                
    print(f"최적의 parameter는 a = {best_a}, b = {best_b}\n이때 평균 에러는 {min_error:.2f} 입니다.")
    return best_a, best_b, min_error

a,b, min_error = find_best_params(a_list, b_list, X, Y)
print("기존 방정식의 평균 에러 : {error:.2f}")
print(f"모델 성능 개선 : {error - min_error:.2f}")

모델 개선이 가능합니다.
a=21, b=0.4
error=539.65873
모델 개선이 가능합니다.
a=21, b=0.5
error=539.64700
모델 개선이 가능합니다.
a=21, b=0.6
error=539.63529
모델 개선이 가능합니다.
a=21, b=0.7
error=539.62359
모델 개선이 가능합니다.
a=21, b=0.8
error=539.61192
모델 개선이 가능합니다.
a=21, b=0.9
error=539.60026
모델 개선이 가능합니다.
a=21.1, b=0.4
error=536.65948
모델 개선이 가능합니다.
a=21.1, b=0.5
error=536.65147
모델 개선이 가능합니다.
a=21.1, b=0.6
error=536.64347
모델 개선이 가능합니다.
a=21.1, b=0.7
error=536.63550
모델 개선이 가능합니다.
a=21.1, b=0.8
error=536.62754
모델 개선이 가능합니다.
a=21.1, b=0.9
error=536.61961
모델 개선이 가능합니다.
a=21.2, b=0.4
error=534.41942
모델 개선이 가능합니다.
a=21.2, b=0.5
error=534.41517
모델 개선이 가능합니다.
a=21.2, b=0.6
error=534.41094
모델 개선이 가능합니다.
a=21.2, b=0.7
error=534.40674
모델 개선이 가능합니다.
a=21.2, b=0.8
error=534.40255
모델 개선이 가능합니다.
a=21.2, b=0.9
error=534.39838
모델 개선이 가능합니다.
a=21.3, b=0.4
error=532.94812
모델 개선이 가능합니다.
a=21.3, b=0.5
error=532.94767
모델 개선이 가능합니다.
a=21.3, b=0.6
error=532.94724
모델 개선이 가능합니다.
a=21.3, b=0.7
error=532.94683
모델 개선이 가능합니다.
a=21.3, b=0.8
error=532.94644


이런 식으로 더 정확한 예측 값을 구하도록 최적의 a와 b라는 best parameter를 찾아가는 과정이 학습입니다.

머신러닝에서는 이 학습 과정이 자동으로 이루어지도록 짜여집니다.

이 방법을 이용해서 더 정확한 방정식을 찾아서 성능을 개선하고,

찾은 방법을 코드 공유 게시판에 올려주세요!

좋아요를 많이 받으시는 분께는 데이콘 후드를 드립니다!

감사합니다!